In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkb, wkt
from shapely.geometry import box, Polygon

import rasterio
import rioxarray as rxa

import matplotlib.pyplot as plt

from datetime import datetime

from metloom.pointdata import SnotelPointData, CDECPointData
from metloom.variables import SnotelVariables, CdecStationVariables

from rasterio import features
def vectorize_valid(fp):
    img = rxa.open_rasterio(fp).squeeze('band')
    img = ~img.isnull()
    img = img.astype('uint8')
    mask = img == 1
    coords = list((features.shapes(img, mask = mask)))[0][0]['coordinates'][0]
    xy_coords = [(img.x[int(x)].values.ravel()[0], img.y[int(y)].values.ravel()[0]) for x, y in coords]
    return Polygon(xy_coords)

fig_dir = Path('/bsuhome/zacharykeskinen/uavsar-coherence/figures/snotels')

uavsar_dir = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar')

uavsars = list(uavsar_dir.glob('*'))
uavsars = [u for u in uavsars if u != 'tmp']
uavsars = uavsars[:2]
uavsars = {'_'.join(u.stem.split('_')[:2]): list(u.glob('*.cor.grd.tiff'))[0] for u in uavsars if len(list(u.glob('*.cor.grd.tiff'))) > 0}
bounds = {u: vectorize_valid(v) for u, v in uavsars.items()}
gpd.GeoDataFrame(bounds.values(), index = bounds.keys(), columns = ['geometry'])

,geometry
grmesa_27416,"POLYGON ((-108.25885 39.16227, -108.25880 39.1..."
stlake_09127,"POLYGON ((-111.68318 40.67539, -111.68241 40.6..."


In [5]:
vrs = [SnotelVariables.SWE, SnotelVariables.SNOWDEPTH, SnotelVariables.TEMPAVG]
snotel_id = '766:UT:SNTL'
snotel_point = SnotelPointData(snotel_id, "my station")
snotel_point.get_daily_data(datetime(2019, 11, 1), datetime(2022, 6, 1), vrs)

,,geometry,SWE,SWE_units,SNOWDEPTH,SNOWDEPTH_units,AVG AIR TEMP,AVG AIR TEMP_units,datasource
datetime,site,,,,,,,,
2019-11-01 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),3.4,in,14.0,in,28.94,degF,NRCS
2019-11-02 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),3.4,in,13.0,in,36.68,degF,NRCS
2019-11-03 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),3.4,in,12.0,in,38.66,degF,NRCS
2019-11-04 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),3.4,in,12.0,in,35.24,degF,NRCS
2019-11-05 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),3.4,in,12.0,in,37.04,degF,NRCS
...,...,...,...,...,...,...,...,...,...
2022-05-28 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),10.0,in,20.0,in,43.52,degF,NRCS
2022-05-29 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),9.4,in,19.0,in,32.00,degF,NRCS
2022-05-30 08:00:00+00:00,766:UT:SNTL,POINT Z (-111.65852 40.56914 9177.00000),10.5,in,21.0,in,30.20,degF,NRCS
